# Proof of a Flat Earth!

- 1. Geostationary orbits match the Earth's rotation:

In [1]:
import requests
import pandas as pd
import time
import folium


In [3]:
MEASUREMENT_ID = 1001

def fetch_ripe_traceroute(measurement_id):
    url = f"https://atlas.ripe.net/api/v2/measurements/{measurement_id}/results/"
    response = requests.get(url, stream=True)
    print("Status:", response.status_code)
    print("Content-Type:", response.headers.get('Content-Type'))
    print("Content-Length:", response.headers.get('Content-Length'))
    response.raise_for_status()
    return response.json()

raw_data = fetch_ripe_traceroute(MEASUREMENT_ID)

Status: 200
Content-Type: application/json
Content-Length: None


KeyboardInterrupt: 

In [ ]:
hops = []
for result in raw_data:
    if 'result' not in result:
        continue
    for hop in result['result']:
        hop_num = hop.get('hop')
        ip = hop.get('from')
        if ip:
            hops.append({'hop': hop_num, 'ip': ip})

df = pd.DataFrame(hops).drop_duplicates().sort_values('hop').reset_index(drop=True)
print(f"Fetched {len(df)} hops")

In [ ]:
def geolocate(ip):
    try:
        r = requests.get(f"http://ip-api.com/json/{ip}", timeout=3)
        d = r.json()
        if d.get("status") == "success":
            return d["lat"], d["lon"], d.get("isp", "Unknown")
    except:
        pass
    return None, None, None

Traceroute hops and IPs:
Empty DataFrame
Columns: [hop, ip]
Index: []


In [ ]:
geo_results = []
for ip in df['ip']:
    geo_results.append(geolocate(ip))
    time.sleep(1.5)

{'status': 'fail',
 'message': 'SSL unavailable for this endpoint, order a key at https://members.ip-api.com/'}

In [ ]:
geo_df = pd.DataFrame(geo_results, columns=['lat', 'lon', 'isp'])
df = pd.concat([df.reset_index(drop=True), geo_df], axis=1)

In [ ]:
valid_hops = df.dropna(subset=['lat', 'lon']).reset_index(drop=True)

if not valid_hops.empty:
    center = [valid_hops.loc[0, 'lat'], valid_hops.loc[0, 'lon']]
    m = folium.Map(location=center, zoom_start=3)

    points = []
    for _, row in valid_hops.iterrows():
        point = (row['lat'], row['lon'])
        points.append(point)
        folium.Marker(
            location=point,
            popup=f"Hop {row['hop']}: {row['ip']}<br>{row['isp']}",
            tooltip=f"Hop {row['hop']}",
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)

    folium.PolyLine(points, color='red', weight=3, opacity=0.8).add_to(m)

    display(m)
else:
    print("No geolocation data found to plot.")